In [1]:
import pandas as pd
import numpy as np

## 인구데이터 전처리
- 행정동에 대한 인구데이터 있음
- 행정동->법정동 변환 규칙 데이터 있음
- 규칙에 따라 행정동에서 법정동 변환 후 같은 법정동끼리 합침

In [21]:
# 인구데이터 처리
df_pop = pd.read_excel("data/population_full.xlsx")
df_pop['발생장소_동'] = df_pop['발생장소_동'].apply(lambda x: x.lstrip())
df_pop['행정동명'] = df_pop['발생장소_시'] + " " + df_pop['발생장소_구'] + " " + df_pop['발생장소_동']
df_pop = df_pop.loc[:, ['행정동명'] + [i for i in range(2014, 2024)]]
df_pop.head(3)

,행정동명,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,충청남도 천안시 동남구 목천읍,27449,27281,26871,26475,25174,24659,24224,24274,24122,23542
1,충청남도 천안시 동남구 풍세면,4689,4599,4481,4390,4312,4255,4165,4107,4080,3981
2,충청남도 천안시 동남구 광덕면,4459,4456,4561,4530,4392,4365,4344,4295,4227,4173


In [27]:
# 행정동명->법정동명을 위한 변환데이터 불러오기
admin_code = pd.read_excel("data/code_var.xlsx")
admin_code = admin_code[admin_code['시도'] == '충청남도']
admin_code = admin_code.loc[admin_code['행정구역코드'].fillna(3333333).apply(lambda x: len(str(int(x))) > 6)].reset_index(drop=True)
admin_code['법정동코드'] = admin_code['법정동코드'].apply(lambda x: str(x)[:-2])
# 읍면동 단위로 정제
admin_code['행정동명'] = admin_code['시도'] + " " + admin_code['시군구'] + " " + admin_code['행정구역명']
admin_code = admin_code.loc[:, ['행정동명', '법정동코드']]
admin_code = admin_code.groupby(['행정동명', '법정동코드']).count().reset_index()
# 오천면어항출장소는 신흑동에 포함되는 것으로 처리
admin_code.loc[admin_code['행정동명'] == '충청남도 보령시 오천면어항출장소', '법정동코드'] = '44180110'
print(len(admin_code))
admin_code.head(3)

300


,행정동명,법정동코드
0,충청남도 계룡시 금암동,44250101
1,충청남도 계룡시 두마면,44250310
2,충청남도 계룡시 신도안면,44250330


In [29]:
# 법정동코드 데이터 불러오기
law_code = pd.read_table("data/법정동코드 전체자료.txt", sep='\t', encoding='cp949')
# 읍면동 단위로 정제
law_code = law_code[law_code['법정동코드'].apply(lambda x: str(x).startswith('44'))]
law_code = law_code[law_code['폐지여부'] == '존재']
law_code = law_code[law_code['법정동명'].apply(lambda x: x[-1] in ('읍', '면', '동'))]
law_code.loc[:, '법정동코드'] = law_code.loc[:, '법정동코드'].apply(lambda x: str(x)[:-2])
law_code.reset_index(drop=True, inplace=True)
law_code.drop(columns='폐지여부', inplace=True)
print(len(law_code))
law_code.head(3)

285


,법정동코드,법정동명
0,44131101,충청남도 천안시 동남구 대흥동
1,44131102,충청남도 천안시 동남구 성황동
2,44131103,충청남도 천안시 동남구 문화동


In [30]:
# 행정동명 테이블과 법정동명 테이블 merge해서 변환테이블 생성
trans_code = pd.merge(admin_code, law_code, how='left', on='법정동코드')
trans_code.head(3)

,행정동명,법정동코드,법정동명
0,충청남도 계룡시 금암동,44250101,충청남도 계룡시 금암동
1,충청남도 계룡시 두마면,44250310,충청남도 계룡시 두마면
2,충청남도 계룡시 신도안면,44250330,충청남도 계룡시 신도안면


In [32]:
# 변환테이블 기반으로 인구데이터 merge
df_pop_law = pd.merge(df_pop, trans_code, how='left', on='행정동명')
print(len(df_pop_law))
df_pop_law.head(3)

300


,행정동명,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,법정동코드,법정동명
0,충청남도 천안시 동남구 목천읍,27449,27281,26871,26475,25174,24659,24224,24274,24122,23542,44131250,충청남도 천안시 동남구 목천읍
1,충청남도 천안시 동남구 풍세면,4689,4599,4481,4390,4312,4255,4165,4107,4080,3981,44131310,충청남도 천안시 동남구 풍세면
2,충청남도 천안시 동남구 광덕면,4459,4456,4561,4530,4392,4365,4344,4295,4227,4173,44131320,충청남도 천안시 동남구 광덕면


In [34]:
# 행정동명이 겹치는 인구데이터 개수 확인
dup = pd.DataFrame(df_pop_law['행정동명'].value_counts()).reset_index()
print(len(dup))
dup.head(3)

209


,행정동명,count
0,충청남도 서산시 석남동,7
1,충청남도 논산시 취암동,7
2,충청남도 공주시 월송동,7


In [35]:
# 중복 행정동에 대해 중복 개수로 평균내기
df_pop_law['중복개수'] = df_pop_law['행정동명'].apply(lambda x: dup.loc[dup['행정동명'] == x, 'count'].values[0])
for i in range(2014, 2024):
    df_pop_law[i] = df_pop_law[i] / df_pop_law['중복개수']
df_pop_law.head(3)

,행정동명,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,법정동코드,법정동명,중복개수
0,충청남도 천안시 동남구 목천읍,27449.0,27281.0,26871.0,26475.0,25174.0,24659.0,24224.0,24274.0,24122.0,23542.0,44131250,충청남도 천안시 동남구 목천읍,1
1,충청남도 천안시 동남구 풍세면,4689.0,4599.0,4481.0,4390.0,4312.0,4255.0,4165.0,4107.0,4080.0,3981.0,44131310,충청남도 천안시 동남구 풍세면,1
2,충청남도 천안시 동남구 광덕면,4459.0,4456.0,4561.0,4530.0,4392.0,4365.0,4344.0,4295.0,4227.0,4173.0,44131320,충청남도 천안시 동남구 광덕면,1


In [37]:
# 행정동 관련 컬럼 삭제
df_pop_law = df_pop_law.drop(columns=['행정동명', '중복개수'])

In [43]:
# 법정동코드가 같은 항목끼리는 인구 합하기
df_pop_law = pd.DataFrame(df_pop_law.groupby(['법정동코드', '법정동명']).sum()).reset_index()
df_pop_law = df_pop_law.loc[:, ['법정동명', '법정동코드'] + [i for i in range(2014, 2024)]]
df_pop_law.head(3)

,법정동명,법정동코드,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,충청남도 천안시 동남구 대흥동,44131101,1498.500000,1486.750000,1464.500000,1409.500000,1426.25,1409.000000,1390.750000,1346.500000,1349.750000,1300.500000
1,충청남도 천안시 동남구 성황동,44131102,1681.000000,1602.333333,1543.000000,1479.333333,1387.00,1305.000000,1226.000000,1435.333333,2214.666667,2628.333333
2,충청남도 천안시 동남구 문화동,44131103,5186.666667,5041.000000,4822.333333,4631.000000,4472.00,4299.666667,4184.666667,4317.000000,5059.333333,5376.666667


In [44]:
# 결측치 및 개수 확인
df_pop_law.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285 entries, 0 to 284
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   법정동명    285 non-null    object 
 1   법정동코드   285 non-null    object 
 2   2014    285 non-null    float64
 3   2015    285 non-null    float64
 4   2016    285 non-null    float64
 5   2017    285 non-null    float64
 6   2018    285 non-null    float64
 7   2019    285 non-null    float64
 8   2020    285 non-null    float64
 9   2021    285 non-null    float64
 10  2022    285 non-null    float64
 11  2023    285 non-null    float64
dtypes: float64(10), object(2)
memory usage: 26.8+ KB


In [45]:
# 전처리 데이터 저장
df_pop_law.to_csv('data/pop_pp.csv', index=False)